In [ ]:
!pip install textattack

In [ ]:
!pip install -U gensim==4.0.0

Requirement already up-to-date: gensim==4.0.0 in /usr/local/lib/python3.7/dist-packages (4.0.0)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FastText Embedding


In [ ]:
from gensim.models.fasttext import FastText
from gensim.test.utils import get_tmpfile  
fname = get_tmpfile("/content/drive/MyDrive/NLP/fasttext.model")
model1 = FastText.load(fname)

Word2Vec Embedding

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.test.utils import get_tmpfile
fname1 = get_tmpfile("/content/drive/MyDrive/NLP/word2vec.model")
model2 = Word2Vec.load(fname1)

Glove Embedding

In [ ]:
from gensim.models import KeyedVectors
model3=KeyedVectors.load('/content/drive/MyDrive/NLP/gloveKeyVectors.kv')

In [ ]:
print(model1.wv.most_similar(positive=['cat']))
print(model2.wv.most_similar(positive=['cat']))
print(model3.most_similar(positive=['cat']))

[('bobcat', 0.759185791015625), ('dog', 0.7208123803138733), ('foxglove', 0.7110776901245117), ('pig', 0.7088143229484558), ('goose', 0.7069101333618164), ('cats', 0.702387273311615), ('beak', 0.7008379697799683), ('catfish', 0.6987316608428955), ('foxhound', 0.696578860282898), ('catnip', 0.6960986852645874)]
[('dog', 0.7193913459777832), ('squirrel', 0.696162760257721), ('leopardus', 0.6945451498031616), ('prionailurus', 0.688106894493103), ('albino', 0.6880608201026917), ('bird', 0.6836060285568237), ('eared', 0.6803243160247803), ('rhinoceros', 0.6769237518310547), ('hyena', 0.6756084561347961), ('badger', 0.6752869486808777)]
[('cats', 0.7006820440292358), ('dog', 0.698471188545227), ('dogs', 0.6442825198173523), ('tervueren', 0.619421660900116), ('laekenois', 0.5972026586532593), ('lica', 0.5956287980079651), ('feral', 0.5899650454521179), ('goat', 0.5861616134643555), ('fish', 0.5746133327484131), ('malinois', 0.5737614631652832)]


In [ ]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-uncased-ag-news")
tokenizer = AutoTokenizer("textattack/distilbert-base-uncased-ag-news")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
print([model1.wv.most_similar(positive=['science'])[0][0]])
print([model2.wv.most_similar(positive=['science'])[0][0]])
print([model3.most_similar(positive=['science'])[0][0]])

['geoscience']
['fiction']
['fiction']


In [ ]:
print([model1.wv.most_similar(positive=['Fears'])[0][0]])
print([model2.wv.most_similar(positive=['Fears'])[0][0]])

['ears']


KeyError: ignored

In [ ]:
from textattack.transformations import WordSwap

class Swapper(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        for i in range(10):
            if model1.wv.most_similar(word)[i][0].lower() == word.lower():
                continue
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()]
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()]
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

In [ ]:
#using word2vec similar word
from textattack.transformations import WordSwap

class Swapper_(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model2.wv.key_to_index:
            for i in range(10):
                if model2.wv.most_similar(word)[i][0].lower() == word.lower():
                    continue
                elif word.isupper():
                    return [model2.wv.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model2.wv.most_similar(word)[i][0].capitalize()]
                else:
                    return [model2.wv.most_similar(word)[i][0]]
        else:
            return [word]

In [ ]:
#using Glove similar word

from textattack.transformations import WordSwap

class Swapper_G(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """ Returns 'banana', no matter what 'word' was originally.

            Returns a list with one item, since `_get_replacement_words` is intended to
                return a list of candidate replacement words.
        """
        if word in model3.key_to_index:
            for i in range(10):
                if model3.most_similar(word)[i][0].lower() == word.lower():
                    continue
                elif word.isupper():
                    return [model3.most_similar(word)[i][0].upper()]
                elif word[0].isupper():
                    return [model3.most_similar(word)[i][0].capitalize()]
                else:
                    return [model3.most_similar(word)[i][0]]
        else:
            return [word]

In [ ]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:47,  2.00s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:11<02:15,  5.88s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:16<01:59,  5.41s/it]

[Succeeded / Failed / Total] 1 / 3 / 4:  16%|█▌        | 4/25 [00:17<01:33,  4.45s/it]

...

[Succeeded / Failed / Total] 6 / 17 / 25: 100%|██████████| 25/25 [03:11<00:00,  7.65s/it]

[Succeeded / Failed / Total] 6 / 18 / 26: : 26it [03:13,  7.43s/it]

[Succeeded / Failed / Total] 6 / 19 / 27: : 27it [03:15,  7.25s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 19     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 70.37% |
| Attack success rate:          | 24.0%  |
| Average perturbed word %:     | 20.38% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 655.4  |
+-------------------------------+--------+


textattack: Attack time: 195.83980298042297s


In [ ]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:12,  1.87it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:02<00:31,  1.35s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:04<00:32,  1.49s/it]

[Succeeded / Failed / Total] 0 / 4 / 4:  16%|█▌        | 4/25 [00:07<00:39,  1.90s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [01:11<00:00,  2.86s/it]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [01:12,  2.79s/it]

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [01:13,  2.71s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 245.2  |
+-------------------------------+--------+


textattack: Attack time: 73.23351311683655s


In [ ]:
from textattack.search_methods import GreedySearch
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_G()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
# We'll use the Greedy search method
search_method = GreedySearch()
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_G
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:29,  1.24s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:07<01:21,  3.52s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:11<01:23,  3.79s/it]

[Succeeded / Failed / Total] 0 / 4 / 4:  16%|█▌        | 4/25 [00:18<01:38,  4.71s/it]

...

[Succeeded / Failed / Total] 3 / 20 / 25: 100%|██████████| 25/25 [02:48<00:00,  6.76s/it]

[Succeeded / Failed / Total] 3 / 21 / 26: : 26it [02:51,  6.61s/it]

[Succeeded / Failed / Total] 3 / 22 / 27: : 27it [02:53,  6.42s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 81.48% |
| Attack success rate:          | 12.0%  |
| Average perturbed word %:     | 10.27% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 252.64 |
+-------------------------------+--------+


textattack: Attack time: 173.26382756233215s


In [ ]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
#seach method
search_method = GreedyWordSwapWIR()
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.41it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:01<00:14,  1.55it/s]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:01<00:14,  1.53it/s]

[Succeeded / Failed / Total] 1 / 3 / 4:  16%|█▌        | 4/25 [00:02<00:12,  1.68it/s]

...

[Succeeded / Failed / Total] 4 / 19 / 25: 100%|██████████| 25/25 [00:16<00:00,  1.53it/s]

[Succeeded / Failed / Total] 4 / 20 / 26: : 26it [00:16,  1.55it/s]

[Succeeded / Failed / Total] 4 / 21 / 27: : 27it [00:17,  1.57it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 21     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 77.78% |
| Attack success rate:          | 16.0%  |
| Average perturbed word %:     | 13.4%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 74.0   |
+-------------------------------+--------+


textattack: Attack time: 17.219042539596558s


In [ ]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
#seach method
search_method = GreedyWordSwapWIR()
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:06,  3.69it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:00<00:09,  2.40it/s]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:01<00:09,  2.25it/s]

[Succeeded / Failed / Total] 0 / 4 / 4:  16%|█▌        | 4/25 [00:01<00:10,  2.04it/s]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [00:12<00:00,  2.04it/s]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [00:12,  2.07it/s]

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [00:12,  2.09it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 63.0   |
+-------------------------------+--------+


textattack: Attack time: 12.89551568031311s


In [ ]:
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_G()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
#seach method
search_method = GreedyWordSwapWIR()
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_G
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:00<00:09,  2.49it/s]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:01<00:15,  1.52it/s]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:02<00:15,  1.43it/s]

[Succeeded / Failed / Total] 0 / 4 / 4:  16%|█▌        | 4/25 [00:03<00:16,  1.29it/s]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [00:19<00:00,  1.32it/s]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [00:19,  1.33it/s]

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [00:20,  1.35it/s]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 63.2   |
+-------------------------------+--------+


textattack: Attack time: 20.04709219932556s


In [ ]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
#seach method
search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True,max_turn_retries=5)
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:06<02:30,  6.28s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:31<05:59, 15.62s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:46<05:38, 15.39s/it]

[Succeeded / Failed / Total] 1 / 3 / 4:  16%|█▌        | 4/25 [00:53<04:41, 13.41s/it]

...

[Succeeded / Failed / Total] 5 / 18 / 25: 100%|██████████| 25/25 [05:59<00:00, 14.38s/it]

[Succeeded / Failed / Total] 5 / 19 / 26: : 26it [06:05, 14.07s/it]

[Succeeded / Failed / Total] 5 / 20 / 27: : 27it [06:13, 13.83s/it]

+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 5       |
| Number of failed attacks:     | 20      |
| Number of skipped attacks:    | 2       |
| Original accuracy:            | 92.59%  |
| Accuracy under attack:        | 74.07%  |
| Attack success rate:          | 20.0%   |
| Average perturbed word %:     | 15.43%  |
| Average num. words per input: | 45.04   |
| Avg num queries:              | 1278.52 |
+-------------------------------+---------+


textattack: Attack time: 373.4261283874512s


In [ ]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
#seach method
search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True,max_turn_retries=5)
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:01<00:35,  1.49s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:08<01:35,  4.15s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:13<01:37,  4.44s/it]

[Succeeded / Failed / Total] 0 / 4 / 4:  16%|█▌        | 4/25 [00:23<02:04,  5.92s/it]

...

[Succeeded / Failed / Total] 2 / 21 / 25: 100%|██████████| 25/25 [03:01<00:00,  7.26s/it]

[Succeeded / Failed / Total] 2 / 22 / 26: : 26it [03:05,  7.13s/it]

[Succeeded / Failed / Total] 2 / 23 / 27: : 27it [03:07,  6.94s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 85.19% |
| Attack success rate:          | 8.0%   |
| Average perturbed word %:     | 3.65%  |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 715.8  |
+-------------------------------+--------+


textattack: Attack time: 187.25276160240173s


In [ ]:
from textattack.search_methods import ParticleSwarmOptimization
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper_G()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(),
               StopwordModification()]
#seach method
search_method = ParticleSwarmOptimization(pop_size=10,max_iters=5,post_turn_check=True,max_turn_retries=5)
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

dataset = HuggingFaceDataset("ag_news", None, "test")

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split test.


In [ ]:
print(attack)

Attack(
  (search_method): ParticleSwarmOptimization(
    (pop_size):  10
    (max_iters):  5
    (post_turn_check):  True
    (max_turn_retries):  20
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper_G
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


In [ ]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML
num_examples = 25
num_remaining_attacks = num_examples
pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

worklist = deque(range(0, num_examples))
worklist_tail = worklist[-1]

attack_log_manager = textattack.loggers.AttackLogManager()

load_time = time.time()
        
num_results = 0
num_failures = 0
num_successes = 0
for result in attack.attack_dataset(dataset, indices=worklist):
    result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
    display(HTML(result_html_str))
    attack_log_manager.log_result(result)
            
    if not isinstance(result, textattack.attack_results.SkippedAttackResult):
        pbar.update(1)
    else:
        worklist_tail += 1
        pbar.update(1)
        worklist.append(worklist_tail)

    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
                
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

pbar.close()

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()
attack_log_manager.flush()
        
textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  0%|          | 0/25 [00:00<?, ?it/s]

[Succeeded / Failed / Total] 0 / 1 / 1:   4%|▍         | 1/25 [00:04<01:55,  4.81s/it]

[Succeeded / Failed / Total] 0 / 2 / 2:   8%|▊         | 2/25 [00:20<03:55, 10.23s/it]

[Succeeded / Failed / Total] 0 / 3 / 3:  12%|█▏        | 3/25 [00:34<04:15, 11.60s/it]

[Succeeded / Failed / Total] 0 / 4 / 4:  16%|█▌        | 4/25 [00:50<04:24, 12.61s/it]

...

[Succeeded / Failed / Total] 3 / 20 / 25: 100%|██████████| 25/25 [06:15<00:00, 15.04s/it]

[Succeeded / Failed / Total] 3 / 21 / 26: : 26it [06:23, 14.76s/it]

[Succeeded / Failed / Total] 3 / 22 / 27: : 27it [06:27, 14.35s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 22     |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 92.59% |
| Accuracy under attack:        | 81.48% |
| Attack success rate:          | 12.0%  |
| Average perturbed word %:     | 12.23% |
| Average num. words per input: | 45.04  |
| Avg num queries:              | 712.92 |
+-------------------------------+--------+


textattack: Attack time: 387.4769477844238s
